# Loop 1 LB Feedback Analysis

Baseline submitted: CV 70.6600 | LB 70.6600 (perfect match!)

Now let's:
1. Check the chistyakov dataset score
2. Scan ALL snapshots for better solutions
3. Identify the best available pre-optimized CSV

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import glob
import os

getcontext().prec = 30

# Parse the 's' prefix from values
def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

# Tree geometry
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0')), float(tip_y)),
            (float(top_w / Decimal('2')), float(tier_1_y)),
            (float(top_w / Decimal('4')), float(tier_1_y)),
            (float(mid_w / Decimal('2')), float(tier_2_y)),
            (float(mid_w / Decimal('4')), float(tier_2_y)),
            (float(base_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(base_y)),
            (float(-(base_w / Decimal('2'))), float(base_y)),
            (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
            (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
            (float(-(top_w / Decimal('4'))), float(tier_1_y)),
            (float(-(top_w / Decimal('2'))), float(tier_1_y)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

print("Scoring utilities loaded.")

In [ ]:
def quick_score(csv_path):
    """Calculate total score for a submission CSV."""
    try:
        df = pd.read_csv(csv_path)
        if len(df) != 20100:
            return None, f"Wrong row count: {len(df)}"
        
        total_score = Decimal('0')
        
        for n in range(1, 201):
            group = df[df['id'].str.startswith(f'{n:03d}_')]
            if len(group) != n:
                return None, f"N={n} has {len(group)} trees"
            
            trees = []
            for _, row in group.iterrows():
                x = parse_value(row['x'])
                y = parse_value(row['y'])
                deg = parse_value(row['deg'])
                trees.append(ChristmasTree(x, y, deg))
            
            polygons = [t.polygon for t in trees]
            bounds = unary_union(polygons).bounds
            side = Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1])))
            score_n = side ** 2 / Decimal(str(n))
            total_score += score_n
        
        return float(total_score), None
    except Exception as e:
        return None, str(e)

print("Quick score function ready.")

In [ ]:
# Check the chistyakov dataset
print("Checking chistyakov dataset...")
chistyakov_path = '/home/code/exploration/datasets/chistyakov/submission_best.csv'
score, error = quick_score(chistyakov_path)
if error:
    print(f"Error: {error}")
else:
    print(f"Chistyakov submission_best.csv score: {score:.6f}")

In [ ]:
# Scan ALL snapshot CSVs for better solutions
print("\nScanning snapshots for better solutions...")
print("This may take a while...\n")

all_csvs = glob.glob('/home/nonroot/snapshots/santa-2025/**/*.csv', recursive=True)
print(f"Found {len(all_csvs)} CSV files in snapshots")

# Filter to likely submission files (correct size)
results = []
for i, csv_path in enumerate(all_csvs):
    if i % 20 == 0:
        print(f"Checking {i}/{len(all_csvs)}...")
    
    # Quick size check first
    try:
        size = os.path.getsize(csv_path)
        # Submission files are typically 1-2MB
        if size < 500000 or size > 3000000:
            continue
    except:
        continue
    
    score, error = quick_score(csv_path)
    if score is not None:
        results.append((score, csv_path))
        if score < 70.66:
            print(f"  FOUND BETTER: {score:.6f} - {csv_path}")

print(f"\nScored {len(results)} valid submission files")

In [ ]:
# Sort and show best results
results.sort(key=lambda x: x[0])

print("\n=== TOP 20 BEST SOLUTIONS ===")
for score, path in results[:20]:
    print(f"{score:.6f} - {path}")

print(f"\n=== SUMMARY ===")
print(f"Best score found: {results[0][0]:.6f}")
print(f"Best file: {results[0][1]}")
print(f"Current baseline: 70.659958")
print(f"Target: 68.919154")
print(f"Gap to target: {results[0][0] - 68.919154:.6f}")

In [ ]:
# Also check the exploration/datasets folder
print("\n=== Checking exploration/datasets folder ===")
for csv_path in glob.glob('/home/code/exploration/datasets/**/*.csv', recursive=True):
    score, error = quick_score(csv_path)
    if score is not None:
        print(f"{score:.6f} - {csv_path}")
    else:
        print(f"Error: {csv_path} - {error}")